In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

import PIL
from PIL import Image

from torch.utils.data.dataset import Dataset
from torchvision import transforms

import pandas as pd 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [5]:
data = np.loadtxt('train-1.txt', dtype=str, delimiter = '\n')
print(data.shape)

(1984694,)


In [9]:
blacks = 0
whites = 0

In [10]:
for elem in data:
    if elem[0] == 'b':
        blacks += 1
    elif elem[0] == 'w':
        whites += 1

In [41]:
blacks, whites

(1019906, 921217)

In [57]:
dataBlack = np.empty(blacks, dtype = '<U718')
dataWhite = np.empty(whites, dtype = '<U718')

In [58]:
dataBlack

array(['', '', '', ..., '', '', ''], dtype='<U718')

In [59]:
curBlack = 0
curWhite = 0

In [60]:
data.dtype

dtype('<U718')

In [61]:
for i in range(data.shape[0]):
    if data[i][0] == 'b':
        dataBlack[curBlack] = data[i]
        curBlack += 1
    elif data[i][0] == 'w':
        dataWhite[curWhite] = data[i]
        curWhite += 1

In [64]:
np.savetxt('dataBlack.txt', dataBlack, delimiter='\n', fmt="%s")

In [65]:
np.savetxt('dataWhite.txt', dataWhite, delimiter='\n', fmt="%s")

In [128]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [142]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:])
    return letter, num

def toBoards(gameBad, color):
    game = gameBad.split()
    res = []
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, len(game)):
        turn = game[i]
        curBoard = np.zeros(shape=(3, 15, 15))
        if curPlayer == 1:
            blackBoard[toTurn(turn)] = 1
        else:
            whiteBoard[toTurn(turn)] = -1
        
        curBoard[1,:] = blackBoard
        curBoard[2,:] = whiteBoard
        if i % 2 == 1 and color == "black":
            curBoard[0,:] = playerBoard
            res.append(curBoard)
        elif i % 2 == 0 and color == "white":
            curBoard[0,:] = playerBoard
            res.append(curBoard)

        curPlayer *= -1
        printBoard(curBoard)
    return np.asarray(res)

In [143]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [144]:
res = toBoards(dataBlack[0], "black") 
print(dataBlack[0])

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1-1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0

In [146]:
res.shape

(10, 3, 15, 15)

In [66]:
class CustomDatasetFromGames(Dataset):
    def __init__(self, img_paths):
        # Read data
        self.data_info = np.load(img_paths)

        # Calculate len
        self.data_len = (self.data_info).shape[0]
        
    def __getitem__(self, index):
        # Get ndarray from index
        img_as_ndarray = self.image_arr[index]
        
        # Open image
        img_as_img = Image.fromarray(img_as_ndarray)
        
        # Reshaping
        
        # Transform image to tensor
        img_as_tensor = torch.from_numpy(np.asarray(img_as_img, dtype=np.float32))
        
        # Preproc
        img_as_tensor = img_as_tensor.unsqueeze(0)

        return img_as_tensor

    def __len__(self):
        return self.data_len


In [67]:
datasetBlack = CustomDatasetFromGames('dataBlack.txt')
trainLoaderBlack = torch.utils.data.DataLoader(dataset=data, batch_size=32, shuffle=False)

OSError: Failed to interpret file 'dataBlack.txt' as a pickle